In [6]:
!pip install clickhouse-connect

In [17]:
from clickhouse_connect import get_client
client = get_client(host='localhost', username='default', password="123")
print(client.query('SELECT 1').result_rows)

[(1,)]


In [18]:
client.command("""
CREATE TABLE trips
(
    `trip_id` UInt32,
    `vendor_id` Enum8('1' = 1, '2' = 2, '3' = 3, '4' = 4, 'CMT' = 5, 'VTS' = 6, 'DDS' = 7, 'B02512' = 10, 'B02598' = 11, 'B02617' = 12, 'B02682' = 13, 'B02764' = 14, '' = 15),
    `pickup_date` Date,
    `pickup_datetime` DateTime,
    `dropoff_date` Date,
    `dropoff_datetime` DateTime,
    `store_and_fwd_flag` UInt8,
    `rate_code_id` UInt8,
    `pickup_longitude` Float64,
    `pickup_latitude` Float64,
    `dropoff_longitude` Float64,
    `dropoff_latitude` Float64,
    `passenger_count` UInt8,
    `trip_distance` Float64,
    `fare_amount` Float32,
    `extra` Float32,
    `mta_tax` Float32,
    `tip_amount` Float32,
    `tolls_amount` Float32,
    `ehail_fee` Float32,
    `improvement_surcharge` Float32,
    `total_amount` Float32,
    `payment_type` Enum8('UNK' = 0, 'CSH' = 1, 'CRE' = 2, 'NOC' = 3, 'DIS' = 4),
    `trip_type` UInt8,
    `pickup` FixedString(25),
    `dropoff` FixedString(25),
    `cab_type` Enum8('yellow' = 1, 'green' = 2, 'uber' = 3),
    `pickup_nyct2010_gid` Int8,
    `pickup_ctlabel` Float32,
    `pickup_borocode` Int8,
    `pickup_ct2010` String,
    `pickup_boroct2010` String,
    `pickup_cdeligibil` String,
    `pickup_ntacode` FixedString(4),
    `pickup_ntaname` String,
    `pickup_puma` UInt16,
    `dropoff_nyct2010_gid` UInt8,
    `dropoff_ctlabel` Float32,
    `dropoff_borocode` UInt8,
    `dropoff_ct2010` String,
    `dropoff_boroct2010` String,
    `dropoff_cdeligibil` String,
    `dropoff_ntacode` FixedString(4),
    `dropoff_ntaname` String,
    `dropoff_puma` UInt16
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(pickup_date)
ORDER BY pickup_datetime;
               """)

DatabaseError: Received ClickHouse exception, code: 57, server response: Code: 57. DB::Exception: Table default.trips already exists. (TABLE_ALREADY_EXISTS) (version 25.10.1.2351 (official build)) (for url http://localhost:8123)

In [8]:
client.command("""INSERT INTO trips
SELECT *
FROM file(
    'trips_*.gz',  -- or use 'trips_*.gz' if both are in same folder
    'TabSeparatedWithNames',
    "
    `trip_id` UInt32,
    `vendor_id` Enum8('1' = 1, '2' = 2, '3' = 3, '4' = 4, 'CMT' = 5, 'VTS' = 6, 'DDS' = 7, 'B02512' = 10, 'B02598' = 11, 'B02617' = 12, 'B02682' = 13, 'B02764' = 14, '' = 15),
    `pickup_date` Date,
    `pickup_datetime` DateTime,
    `dropoff_date` Date,
    `dropoff_datetime` DateTime,
    `store_and_fwd_flag` UInt8,
    `rate_code_id` UInt8,
    `pickup_longitude` Float64,
    `pickup_latitude` Float64,
    `dropoff_longitude` Float64,
    `dropoff_latitude` Float64,
    `passenger_count` UInt8,
    `trip_distance` Float64,
    `fare_amount` Float32,
    `extra` Float32,
    `mta_tax` Float32,
    `tip_amount` Float32,
    `tolls_amount` Float32,
    `ehail_fee` Float32,
    `improvement_surcharge` Float32,
    `total_amount` Float32,
    `payment_type` Enum8('UNK' = 0, 'CSH' = 1, 'CRE' = 2, 'NOC' = 3, 'DIS' = 4),
    `trip_type` UInt8,
    `pickup` FixedString(25),
    `dropoff` FixedString(25),
    `cab_type` Enum8('yellow' = 1, 'green' = 2, 'uber' = 3),
    `pickup_nyct2010_gid` Int8,
    `pickup_ctlabel` Float32,
    `pickup_borocode` Int8,
    `pickup_ct2010` String,
    `pickup_boroct2010` String,
    `pickup_cdeligibil` String,
    `pickup_ntacode` FixedString(4),
    `pickup_ntaname` String,
    `pickup_puma` UInt16,
    `dropoff_nyct2010_gid` UInt8,
    `dropoff_ctlabel` Float32,
    `dropoff_borocode` UInt8,
    `dropoff_ct2010` String,
    `dropoff_boroct2010` String,
    `dropoff_cdeligibil` String,
    `dropoff_ntacode` FixedString(4),
    `dropoff_ntaname` String,
    `dropoff_puma` UInt16
    "
) SETTINGS input_format_try_infer_datetimes = 0;
               """)

In [10]:
client.command("""SELECT COUNT() FROM trips""")

1999657

In [11]:
client.command("""
    SELECT round(avg(tip_amount), 2) FROM trips
    """)

'1.68'

In [12]:
client.command("""
SELECT
    passenger_count,
    ceil(avg(total_amount),2) AS average_total_amount
FROM trips
GROUP BY passenger_count
               """)

['0',
 '22.69\n1',
 '15.97\n2',
 '17.15\n3',
 '16.76\n4',
 '17.33\n5',
 '16.35\n6',
 '16.04\n7',
 '59.8\n8',
 '36.41\n9',
 '9.81']

In [13]:
client.command("""
SELECT
    pickup_date,
    pickup_ntaname,
    SUM(1) AS number_of_trips
FROM trips
GROUP BY pickup_date, pickup_ntaname
ORDER BY pickup_date ASC""")

['2015-07-01',
 'Melrose South-Mott Haven North',
 '1\n2015-07-01',
 'park-cemetery-etc-Manhattan',
 '169\n2015-07-01',
 'Bedford',
 '8\n2015-07-01',
 'South Jamaica',
 '1\n2015-07-01',
 'Upper West Side',
 '593\n2015-07-01',
 'Borough Park',
 '1\n2015-07-01',
 'Midtown-Midtown South',
 '2258\n2015-07-01',
 'Turtle Bay-East Midtown',
 '999\n2015-07-01',
 'East Village',
 '337\n2015-07-01',
 'Brooklyn Heights-Cobble Hill',
 '13\n2015-07-01',
 'Crown Heights North',
 '3\n2015-07-01',
 'Central Harlem South',
 '33\n2015-07-01',
 '',
 '205\n2015-07-01',
 'North Side-South Side',
 '31\n2015-07-01',
 'Woodside',
 '8\n2015-07-01',
 'Elmhurst',
 '4\n2015-07-01',
 'Flatbush',
 '2\n2015-07-01',
 'Prospect Heights',
 '5\n2015-07-01',
 'Newark Airport',
 '1\n2015-07-01',
 'Old Astoria',
 '5\n2015-07-01',
 'Bushwick North',
 '2\n2015-07-01',
 'Battery Park City-Lower Manhattan',
 '363\n2015-07-01',
 'Midwood',
 '2\n2015-07-01',
 'Forest Hills',
 '5\n2015-07-01',
 'Prospect Lefferts Gardens-Wingate'

In [14]:
client.command("""
SELECT
    avg(tip_amount) AS avg_tip,
    avg(fare_amount) AS avg_fare,
    avg(passenger_count) AS avg_passenger,
    count() AS count,
    truncate(date_diff('second', pickup_datetime, dropoff_datetime)/60) as trip_minutes
FROM trips
WHERE trip_minutes > 0
GROUP BY trip_minutes
ORDER BY trip_minutes DESC""")

['1.9600000381469727',
 '8',
 '1',
 '1',
 '27511\n0',
 '12',
 '2',
 '1',
 '27500\n0.542166673981895',
 '19.716666666666665',
 '1.9166666666666667',
 '60',
 '1439\n0.902499997522682',
 '11.270625001192093',
 '1.95625',
 '160',
 '1438\n0.9715789457909146',
 '13.646616541353383',
 '2.0526315789473686',
 '133',
 '1437\n0.9682692398245518',
 '14.134615384615385',
 '2.076923076923077',
 '104',
 '1436\n1.1022105210705808',
 '13.778947368421052',
 '2.042105263157895',
 '95',
 '1435\n0.8419117699651157',
 '12.441176470588236',
 '2.3529411764705883',
 '68',
 '1434\n0.894264710300109',
 '13.242647058823529',
 '2.411764705882353',
 '68',
 '1433\n0.8565151565002672',
 '15.992424242424242',
 '2.212121212121212',
 '66',
 '1432\n0.7527777817514207',
 '14.592592592592593',
 '1.8333333333333333',
 '54',
 '1431\n1.5949999883770942',
 '14.9375',
 '2.4',
 '40',
 '1430\n1.5668965393099292',
 '15.379310344827585',
 '1.6206896551724137',
 '29',
 '1429\n1.1254838974245134',
 '12.451612903225806',
 '2.322580645

In [15]:
client.command("""
SELECT
    pickup_ntaname,
    toHour(pickup_datetime) as pickup_hour,
    SUM(1) AS pickups
FROM trips
WHERE pickup_ntaname != ''
GROUP BY pickup_ntaname, pickup_hour
ORDER BY pickup_ntaname, pickup_hour""")

['Airport',
 '0',
 '3509\nAirport',
 '1',
 '1184\nAirport',
 '2',
 '401\nAirport',
 '3',
 '152\nAirport',
 '4',
 '213\nAirport',
 '5',
 '955\nAirport',
 '6',
 '2161\nAirport',
 '7',
 '3013\nAirport',
 '8',
 '3601\nAirport',
 '9',
 '3792\nAirport',
 '10',
 '4546\nAirport',
 '11',
 '4659\nAirport',
 '12',
 '4621\nAirport',
 '13',
 '5348\nAirport',
 '14',
 '5889\nAirport',
 '15',
 '6505\nAirport',
 '16',
 '6119\nAirport',
 '17',
 '6341\nAirport',
 '18',
 '6173\nAirport',
 '19',
 '6329\nAirport',
 '20',
 '6271\nAirport',
 '21',
 '6649\nAirport',
 '22',
 '6356\nAirport',
 '23',
 '6016\nAllerton-Pelham Gardens',
 '4',
 '1\nAllerton-Pelham Gardens',
 '6',
 '1\nAllerton-Pelham Gardens',
 '7',
 '1\nAllerton-Pelham Gardens',
 '9',
 '5\nAllerton-Pelham Gardens',
 '10',
 '3\nAllerton-Pelham Gardens',
 '15',
 '1\nAllerton-Pelham Gardens',
 '20',
 '2\nAllerton-Pelham Gardens',
 '23',
 "1\nAnnadale-Huguenot-Prince\\'s Bay-Eltingville",
 '23',
 '1\nArden Heights',
 '11',
 '1\nAstoria',
 '0',
 '287\nAs

In [16]:
client.command("""
SELECT
    pickup_datetime,
    dropoff_datetime,
    total_amount,
    pickup_nyct2010_gid,
    dropoff_nyct2010_gid,
    CASE
        WHEN dropoff_nyct2010_gid = 138 THEN 'LGA'
        WHEN dropoff_nyct2010_gid = 132 THEN 'JFK'
    END AS airport_code,
    EXTRACT(YEAR FROM pickup_datetime) AS year,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    EXTRACT(HOUR FROM pickup_datetime) AS hour
FROM trips
WHERE dropoff_nyct2010_gid IN (132, 138)
ORDER BY pickup_datetime""")

['2015-07-01 00:04:14',
 '2015-07-01 00:15:29',
 '13.3',
 '-34',
 '132',
 'JFK',
 '2015',
 '1',
 '0\n2015-07-01 00:09:42',
 '2015-07-01 00:12:55',
 '6.8',
 '50',
 '138',
 'LGA',
 '2015',
 '1',
 '0\n2015-07-01 00:23:04',
 '2015-07-01 00:24:39',
 '4.8',
 '-125',
 '132',
 'JFK',
 '2015',
 '1',
 '0\n2015-07-01 00:27:51',
 '2015-07-01 00:39:02',
 '14.72',
 '-101',
 '138',
 'LGA',
 '2015',
 '1',
 '0\n2015-07-01 00:32:03',
 '2015-07-01 00:55:39',
 '39.34',
 '48',
 '138',
 'LGA',
 '2015',
 '1',
 '0\n2015-07-01 00:34:12',
 '2015-07-01 00:40:48',
 '9.95',
 '-93',
 '132',
 'JFK',
 '2015',
 '1',
 '0\n2015-07-01 00:38:26',
 '2015-07-01 00:49:00',
 '13.3',
 '-11',
 '138',
 'LGA',
 '2015',
 '1',
 '0\n2015-07-01 00:41:48',
 '2015-07-01 00:44:45',
 '6.3',
 '-94',
 '132',
 'JFK',
 '2015',
 '1',
 '0\n2015-07-01 01:06:18',
 '2015-07-01 01:14:43',
 '11.76',
 '37',
 '132',
 'JFK',
 '2015',
 '1',
 '1\n2015-07-01 01:07:34',
 '2015-07-01 01:30:56',
 '23.8',
 '42',
 '138',
 'LGA',
 '2015',
 '1',
 '1\n2015-07-01

In [16]:
# Create a sample database and table
client.command("CREATE DATABASE IF NOT EXISTS demo")
client.command("""
CREATE TABLE IF NOT EXISTS demo.sales (
    id UInt32,
    region String,
    amount Float64
) ENGINE = MergeTree()
ORDER BY id
""")
print("Table 'demo.sales' created successfully.")

Table 'demo.sales' created successfully.


In [17]:
# Insert sample data into the table
data = [(1, 'EU', 1200.0), (2, 'ASIA', 800.0), (3, 'EU', 500.0), (4, 'NA', 2000.0)]
client.insert('demo.sales', data, column_names=['id', 'region', 'amount'])
print("Inserted sample rows into demo.sales.")

Inserted sample rows into demo.sales.


In [18]:
# Run a query to retrieve all data
result = client.query_df("SELECT * FROM demo.sales")
print(result)

   id region  amount
0   1     EU  1200.0
1   2   ASIA   800.0
2   3     EU   500.0
3   4     NA  2000.0


In [19]:
# Aggregation query - total sales by region
agg_result = client.query_df("SELECT region, SUM(amount) AS total_sales FROM demo.sales GROUP BY region ORDER BY total_sales DESC")
print(agg_result)

  region  total_sales
0     NA       2000.0
1     EU       1700.0
2   ASIA        800.0


In [20]:
# Filtering query - show regions with total sales > 1000
filter_result = client.query_df("SELECT region, SUM(amount) AS total_sales FROM demo.sales GROUP BY region HAVING total_sales > 1000")
print(filter_result)

  region  total_sales
0     NA       2000.0
1     EU       1700.0


In [13]:
# Clean up
client.command("DROP TABLE IF EXISTS demo.sales")
client.command("DROP DATABASE IF EXISTS demo")
print("Cleaned up demo database.")


Cleaned up demo database.


In [24]:
# Aggregating using built-in functions
func_df = client.query_df("""
                          SELECT region,
                                 COUNT()     AS transactions,
                                 AVG(amount) AS avg_sale,
                                 MAX(amount) AS max_sale,
                                 MIN(amount) AS min_sale
                          FROM demo.sales
                          GROUP BY region
                          ORDER BY avg_sale DESC
                          """)
print(func_df)

  region  transactions  avg_sale  max_sale  min_sale
0     NA             1    2000.0    2000.0    2000.0
1     EU             2     850.0    1200.0     500.0
2   ASIA             1     800.0     800.0     800.0


In [23]:
# Query using string functions
string_df = client.query_df("""
                            SELECT region, length(region) AS len, upper(region) AS upper_region
                            FROM demo.sales
                            ORDER BY len DESC
                            """)
print(string_df)

  region  len upper_region
0   ASIA    4         ASIA
1     EU    2           EU
2     EU    2           EU
3     NA    2           NA


In [22]:
# Demonstrate join operation
client.command("""
               CREATE TABLE IF NOT EXISTS demo.regions_info
               (
                   region
                   String,
                   continent
                   String
               ) ENGINE = MergeTree
               (
               )
                   ORDER BY region
               """)
client.insert('demo.regions_info', [('EU', 'Europe'), ('ASIA', 'Asia'), ('NA', 'North America')],
              column_names=['region', 'continent'])

join_df = client.query_df("""
                          SELECT s.region, r.continent, SUM(s.amount) AS total
                          FROM demo.sales s
                                   JOIN demo.regions_info r ON s.region = r.region
                          GROUP BY s.region, r.continent
                          ORDER BY total DESC
                          """)
print(join_df)

  region      continent   total
0     NA  North America  2000.0
1     EU         Europe  1700.0
2   ASIA           Asia   800.0
